# Agent Module — Test Notebook

Test the RAG agent pipeline: prompt loading, memory management, chat completion, and the full `run()` function.

Each section can be run independently.

In [ ]:
import sys
from pathlib import Path

# Ensure the project root is on the path
PROJECT_ROOT = Path.cwd().parent if Path.cwd().name == "agent" else Path.cwd()
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

print(f"Project root: {PROJECT_ROOT}")

## 1. Load System Prompt

Test loading prompts from text, docx, or using a default.

In [ ]:
from agent.prompt import load_prompt

# Load from the default built-in prompt
default_prompt = load_prompt(None)
print("Default prompt:")
print(default_prompt)

In [ ]:
# Load from a custom string fallback
custom_prompt = load_prompt(None, default="You are a friendly shopping assistant for Jaded Rose.")
print("Custom prompt:")
print(custom_prompt)

In [ ]:
# Load from the system_prompt.txt file
prompt_path = PROJECT_ROOT / "agent" / "system_prompt.txt"
if prompt_path.exists():
    file_prompt = load_prompt(prompt_path)
    print(f"Loaded from {prompt_path}:")
    print(file_prompt[:200], "..." if len(file_prompt) > 200 else "")
else:
    print(f"File not found: {prompt_path}")

## 2. Memory Management

Test saving, loading, and clearing conversation memory.

In [ ]:
from agent.memory import load_memory, save_memory, clear_memory, append_exchange, get_pair_count
import tempfile, os

# Use a temp file so we don't overwrite real memory
test_memory_path = os.path.join(tempfile.gettempdir(), "test_memory.json")

# Start with empty memory
history = load_memory(test_memory_path)
print(f"Initial history: {history}")
print(f"Pair count: {get_pair_count(history)}")

In [ ]:
# Add some exchanges
history = append_exchange(history, "What is your return policy?", "You can return items within 30 days.")
history = append_exchange(history, "Do you ship internationally?", "Yes, we ship to over 50 countries.")

print(f"History has {get_pair_count(history)} exchange(s):")
for msg in history:
    print(f"  [{msg['role']}] {msg['content'][:60]}")

In [ ]:
# Save and reload
save_memory(test_memory_path, history, max_pairs=10)

reloaded = load_memory(test_memory_path)
print(f"Reloaded {get_pair_count(reloaded)} exchange(s) from disk")
assert reloaded == history, "Memory round-trip failed!"
print("Round-trip OK")

In [ ]:
# Clear memory
clear_memory(test_memory_path)
print(f"After clear: {load_memory(test_memory_path)}")

## 3. Chat Completion (build_messages)

Test message construction without making API calls.

In [ ]:
from agent.chat import build_messages

# Example inputs
system_prompt = "You are a helpful shopping assistant."
context = "[1] Returns are accepted within 30 days.\n\n[2] Free shipping on orders over $50."
history = [
    {"role": "user", "content": "Hi there!"},
    {"role": "assistant", "content": "Hello! How can I help you today?"},
]
question = "What is your return policy?"

messages = build_messages(system_prompt, context, history, question)

print(f"Total messages: {len(messages)}\n")
for i, msg in enumerate(messages):
    print(f"[{i}] role={msg['role']}")
    print(f"    {msg['content'][:100]}..." if len(msg['content']) > 100 else f"    {msg['content']}")
    print()

In [ ]:
# Without context — context message should be omitted
messages_no_ctx = build_messages(system_prompt, "", [], "Hello")
print(f"Messages without context: {len(messages_no_ctx)}")
for msg in messages_no_ctx:
    print(f"  [{msg['role']}] {msg['content'][:80]}")

## 4. Full Agent Run (requires API keys)

This cell calls the live OpenAI and Pinecone APIs.
Make sure your `config.json` is set up before running.

In [ ]:
from agent.agent import run

# Direct function call — pass a question, get an answer
question = "What products do you sell?"

try:
    answer = run(question)
    print(f"Q: {question}")
    print(f"A: {answer}")
except FileNotFoundError as e:
    print(f"Config not found (expected if no config.json): {e}")
except Exception as e:
    print(f"Error: {e}")

In [ ]:
# Follow-up question (uses memory from the previous call)
followup = "Can I get a discount on those?"

try:
    answer2 = run(followup)
    print(f"Q: {followup}")
    print(f"A: {answer2}")
except FileNotFoundError as e:
    print(f"Config not found: {e}")
except Exception as e:
    print(f"Error: {e}")

In [ ]:
# Using a custom system prompt override
try:
    answer3 = run(
        "Tell me a joke",
        system_prompt_override="You are a comedian. Always respond with a joke.",
    )
    print(f"A: {answer3}")
except FileNotFoundError as e:
    print(f"Config not found: {e}")
except Exception as e:
    print(f"Error: {e}")